In [ ]:
import numpy as np
from scipy import signal
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
import pyxdf
import glob
import os
from pyedflib import highlevel
import config
from utils.freq_calculator import do_bandpass
from utils.edf_convert import edf_info_single_channel

### Initialise constants

In [ ]:
folder = "/Users/eliasmeier/Documents/IDUN/Data/Eurostars_Cerebra/ORP_inear_adaptation/02_Pilot_study"
subject = "S006"
night = "night4"
idun_file_ending = "synced_data"

freq = [0.3,45]
channels_to_filter = ["IDUN"]
upscale_factor = 1e6
channels_to_upscale = ["CHIN","LEFT_EEG","LEFT_EYE","RIGHT_EYE","RIGHT_EEG","MASTOID"]
channels_to_remove = ["AUDIO","POSITION"]
voltage_units = ['uV','uV','uV','uV','uV','uV','uV'] # ['','uV','uV','uV','uV','','uV','uV','uV'] AUDIO and POSITION would need no unit
clip_limits = [-500,500]
patient_additional = 'IDUN Technologies'
rec_type = 'Sleep recording'

csv_file_path = glob.glob(
    os.path.join(folder, subject, night, f"*{idun_file_ending}.csv")
)[0]


### Upload data

The synced data output from the sync_main stores the data as a dataframe

In [ ]:
# load complete data set
data_all = pd.read_csv(csv_file_path,index_col=0)

data_all.head()

### Remove data

Some channels may not make sense for the edf analysis so it is best to remove them

In [ ]:
pruned_data = data_all.drop(columns=channels_to_remove)

### Filter data

It is important to filter data because EDF only has 16 bits to represent the signal. If we don't filter the data, the signal will be distorted. This is because if there is a DC shift of the signal, the EDF converter is forced to reduce the resolution of the signal to represent the DC shift.

In [ ]:
preprocessed_data = pruned_data.copy()
for channel in channels_to_filter:
    raw_eeg = np.array(data_all[channel].values)
    filtered_eeg = do_bandpass(raw_eeg, freq, sample_rate=config.BASE_SAMPLE_RATE)
    preprocessed_data[channel] = filtered_eeg

preprocessed_data.head()

### Scale data

Some of the data may not be in the same scale, such as in this example the external dataset is in volts while the idun data is in microvolts. We need to scale the data to the same scale.

In [ ]:
scaled_preprocessed_data = preprocessed_data.copy()
for channel in channels_to_upscale:
    scaled_preprocessed_data[channel] = preprocessed_data[channel] * upscale_factor

scaled_preprocessed_data.head()

### Removed too high spikes

The EDF converter cannot handle too high spikes. We need to remove them. Therefore any data above 1000 microvolts is made equal to 1000 microvolts. The same with the negative values.

In [ ]:
# clip data
clipped_data = scaled_preprocessed_data.copy()
all_channels = list(clipped_data.columns)
for channel in all_channels:
    clipped_data[channel] = clipped_data[channel].clip(
        lower=clip_limits[0], upper=clip_limits[1]
    )


### Visualise data

This is for sanity check. We can see if the data is in the correct scale and if the data is filtered correctly.

Notice how I re-reference the data so that it shows similar signals as IDUN

In [ ]:
# plot LEFT_EEG, RIGHT_EEG, IDUN
fig, ax = plt.subplots(1, 1, figsize=(20, 5))
time_axis_index = clipped_data.index.values / config.BASE_SAMPLE_RATE
ax.plot(time_axis_index,clipped_data["IDUN"].values, label="IDUN")
ax.plot(time_axis_index,clipped_data["RIGHT_EEG"].values-clipped_data["LEFT_EEG"].values, label="External EEG")
plt.title("IDUN and external EEG")
plt.legend()
plt.ylabel("Voltage (uV)")
plt.xlabel("Time (s)")
plt.ylim(-600, 600)

### Do EDF conversion

In [ ]:
# Get Channel names
ch_names = list(clipped_data.columns)
print(ch_names)

# Transpose data
clipped_data_values = clipped_data.values
clipped_data_tr = np.transpose(clipped_data_values)
print(np.shape(clipped_data_tr))

# Create saving name
new_savename = os.path.join(folder, subject, night,'eeg_data.edf')

# prepare edf info 
signal_info = []
for chan_indx in range(len(ch_names)):
    add_channel = edf_info_single_channel(clipped_data_tr[chan_indx], ch_names[chan_indx], config.BASE_SAMPLE_RATE, voltage_units[chan_indx])
    signal_info.append(add_channel)
    
# create header for edf conversioon
data_header = highlevel.make_header(patientname=subject, patient_additional=patient_additional, recording_additional=rec_type)
# save signal as edf
highlevel.write_edf(new_savename, clipped_data_tr, signal_info, data_header)